---
# 10: Hidden Markov Models
---

### Intro: finding CpG-islands 

CpG dinucleotides are the least common due to cytosine methylation increasing the rate of deamination (C->T).
So, dinucleotide CG (reverse complement is also CG, btw) is depleted in many genomes. However, methylated is suppresed around genes in 'CpG islands'. We can think of CpG island (or not) as a hidden state in a HMM.


#### HMM probabilities:

Since coin flips are independent events, the probability that $n$ flips of the fair coin will generate a given sequence $x = x_1, x_2, ..., x_n $ with $k$ occurrences of “$Heads$” is

$ \mathrm{Pr}(x|\textit{F}) = \displaystyle\prod_{i=1}^{n}\mathrm{Pr}_F(x_i) = \left(1/2\right)^n $

On the other hand, the probability that a ($p(H)=1/4$) biased coin will generate the same sequence is 

$ \mathrm{Pr}(x|B) =\displaystyle\prod_{i=1}^{n}\mathrm{Pr}_B(x_i) = \left(1/4\right)^{n-k} \cdot \left(3/4\right)^k = 3^k/4^n $

-  **If $Pr(x|F) > Pr(x|B)$**, then the dealer more likely used a fair coin, 
-  else if $Pr(x|F) < Pr(x|B)$, then the dealer more likely used a biased coin. 


<br>

The numbers $(1/2)^n$ and $3^k/4^n$ are so small for large n that in order to compare them, we will use their **log-odds ratio**:  

<br>  

$ \log_2{\left(\dfrac{\mathrm{Pr}(x|F)}{\mathrm{Pr}(x|B)}\right)} = \log_2{\left(\dfrac{2^n}{3^k}\right)} = n - k \cdot \log_2{3} $

<br>
 
**Exercise Break**: Show that Pr(x|F) is larger than Pr(x|B) when the log-odds ratio is positive (i.e., when k/n < 1/ log2(3)) and smaller than Pr(x|B) when the log-odds ratio is negative (i.e., when k/n > 1/ log2(3)).
<br>

**note** big $\displaystyle\prod_{i=1}^{n}$ means the product of probabilites $\mathrm{Pr}_{State}(x_i)$.
Recall, multiple independent events in sequence (1 and 2 and...) -> use *'the product rule'*

---

<br>

Each **emitted string $x$ has probability $Pr(x)$, which is *independent of the hidden path* taken by the HMM**:

$ \mathrm{Pr}(x) = \displaystyle\sum_{\text{all hidden paths } \pi} \mathrm{Pr}(x, \pi)$

Each hidden path π has probability Pr(π), which is independent of the string that the HMM emits:

$\mathrm{Pr}(\pi) = \displaystyle\sum_{\text{all~strings~of~emitted~symbols } x}\mathrm{Pr}(x, \pi)$

The event “the HMM follows the hidden path π and emits x” can be thought of as a combination of two consecutive events:

- The HMM follows the path π. The probability of this event is Pr(π).
- The HMM emits x, given that the HMM follows the path π. 
  We refer to the probability of this event as the conditional probability of x given π, denoted Pr(x|π).
- Both of these events must occur for the HMM to follow path π and emit string x, which implies that

$ Pr(x, π) = Pr(x|π) · Pr(π) $

<br>
<br>

### BA10A: Compute the Probability of a Hidden Path 

---

**Probability of a Hidden Path Problem**  

*Given*: 
    A hidden path $π$ followed by the states States and transition matrix *Transition* of an HMM (Σ, States, Transition, Emission).

*Return*: 
    The probability of this path, $Pr(π)$. You may assume that initial probabilities are equal.  

Probability of a Hidden Path Problem: Compute the probability of a hidden path.

**Input**: A hidden path π in an HMM (Σ, States, Transition, Emission).
**Output**: The probability of this path, Pr(π).

Sample Input:

    ABABBBAAAA
    --------
    A B
    --------
            A	B
        A	0.377	0.623
        B	0.26	0.74

---

In [1]:
import numpy as np

def p_hiddenpath(hidden_path, T_matrix, states):
    """Calculates the probability of a hiddenpath given a T_matrix HMM"""
    # parse hidden path to integers corresponding to E, T matrices
    hidden_path = [int(states.index(state)) for state in hidden_path]

    # assume transition from the initial state occur with equal probability.
    probability = 0.5
    prev = hidden_path[0]
    for state in hidden_path[1:]:
#         print(state, '->', prev)
#         print('trans_p = ', trans_probability)
#         print('prob = ;', probability)
        trans_probability = T_matrix[prev][state]
        probability = probability * trans_probability
        prev = state
    
    return probability

In [2]:
with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/10a_test.txt") as f:
    lines = [line.strip() for line in f]
    hidden_path = lines[0].strip()
    states = lines[2].split()
    T_matrix = np.array([line.split()[1:] for line in lines[5:]], float)
    
print(T_matrix, hidden_path, states)

[[0.377 0.623]
 [0.26  0.74 ]] ABABBBAAAA ['A', 'B']


In [3]:
print("Pr(hidden_path) =", p_hiddenpath(hidden_path, T_matrix, states))

Pr(hidden_path) = 0.0003849286917546758


Correct answer is: 
0.0003849286917546758

---

In [4]:
with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/dataset_26255_8.txt") as f:
    lines = [line.strip() for line in f]
    hidden_path = lines[0].strip()
    states = lines[2].split()
    T_matrix = np.array([line.split()[1:] for line in lines[5:]], float)
    
p_hiddenpath(hidden_path, T_matrix, states)

7.955720786186205e-16

<br>
Good news for you, correct! (answer for test data: 7.955720786186205e-16)
<br>

 ---
 
Note that we have already computed $Pr(x|π)$ for the **crooked dealer HMM** when the dealer’s hidden path consisted only of B or F (fair or biased coin), which we wrote as $Pr(x|B)$ and $Pr(x|F)$, respectively (the probabilities of heads given biased or fair coins).   

To compute $Pr(x|π)$ for a general HMM, we will write $Pr(x_i|π_i)$ to denote the emission probability $emission_{π_i}(x_i) $ that symbol $x_i$ was emitted given that the HMM was in state $π_i$.  

As a result, for a given path $π$, the HMM emits a string $x$ with probability equal to the product of emission probabilities along that path,

---

### BA10B: probability of emission seq given hidden path and emission matrix

<br>

**Input**: A string x, followed by the alphabet from which x was constructed, followed by a hidden path π, followed by the states States and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

<br> 

**Output**: The conditional probability Pr(x|π) that x will be emitted given that the HMM follows the hidden path π.

<br>

Sample Input:

    zzzyxyyzzx                    # emission string x
    --------
    x y z                         # emission alphabet
    --------
    BAAAAAAAAA                    # hiddenpath pi
    -------- 
    A B                           # hidden States
    --------
        x	y	z
    A	0.176	0.596	0.228
    B	0.225	0.572	0.203     # Emission matrix


Sample Output:

    3.59748954746e-06

In [72]:
def p_emissions(emissions, alphabet, hidden_path, states, e_matrix):
    """ calculate probability of this emission string given that hidden_path and Ematrix """
    hidden_path = [int(states.index(state)) for state in hidden_path]
    emissions = [int(alphabet.index(em)) for em in emissions]
    probability = 1
    for i in range(len(emissions)):
        probability = probability * e_matrix[hidden_path[i]][emissions[i]]
    print(probability)

In [73]:
with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/10b_test.txt") as f:
    lines = [line.strip() for line in f]
    emissions = lines[0].strip()
    alphabet = lines[2].strip().split()
    hidden_path = lines[4].strip()
    states = lines[6].strip().split()
    e_matrix = np.array([line.split()[1:] for line in lines[9:]], float)
    
print('string:', emissions,'alphabet', alphabet,'\n', 'hidden_path', hidden_path,'states',states,'\n\n', 'e_matrix','\n',e_matrix, '\n')
print('\nSolution:')
p_emissions(emissions, alphabet, hidden_path, states, e_matrix)

string: zzzyxyyzzx alphabet ['x', 'y', 'z'] 
 hidden_path BAAAAAAAAA states ['A', 'B'] 

 e_matrix 
 [[0.176 0.596 0.228]
 [0.225 0.572 0.203]] 


Solution:
3.5974895474624624e-06


In [74]:
# test data

with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/dataset_26255_10.txt") as f:
    lines = [line.strip() for line in f]
    emissions = lines[0].strip()
    alphabet = lines[2].strip().split()
    hidden_path = lines[4].strip()
    states = lines[6].strip().split()
    e_matrix = np.array([line.split()[1:] for line in lines[9:]], float)
    
# print('string:', emissions,'\n', 'alphabet', alphabet,'\n\n\n', 'hidden_path', hidden_path,'\n\n',
#       'states',states,'\n\n', 'e_matrix','\n',e_matrix, '\n\n\n')
print('\nSolution:')
p_emissions(emissions, alphabet, hidden_path, states, e_matrix)


Solution:
1.1523621771156757e-26


---

### BA10C: Viterbi Algorithm for max-likelihood hidden path


Returning to our formula for Pr(x, π):  
the **probability that an HMM follows path π and emits string x** can be written as a product of emission and transition probabilities:

$ \begin{aligned} \mathrm{Pr}(x, \pi) & = \color{blue}{\mathrm{Pr}(x|\pi)} \cdot \color{green}{\mathrm{Pr}(\pi)}\\ & = \displaystyle\prod_{i=1}^{n} {\,\color{blue}{\mathrm{Pr}(x_i|\pi_i)} \cdot \color{green}{\mathrm{Pr}(\pi_{i-1} \rightarrow \pi_i)}}\\ & = \displaystyle\prod_{i=1}^{n}{\,\color{purple}{\textit{emission}_{\pi_i}(x_i)} \cdot \color{crimson}{\textit{transition}_{\pi_{i-1},\,\pi_i}}}\,. \end{aligned} $
 

Exercise Break: Compute Pr(x, π) for the x and π in the reproduced figure below. Can you find a better explanation for x = “THTHHHTHTTH” than π = FFFBBBBBFFF?

STOP and Think: Now that you have learned about HMMs, try designing an HMM that will model searching for CG-islands in a genome. What barriers do you encounter? 
-  how to define what a CG-island is: what is the emisson prob of CGisland vs not?
<br>

---

<br>

#### HMM -> The decoding problem and Viterbi algorithm


As we stated in the previous section, in both the crooked dealer and CG-island HMMs, we are **looking for the most likely hidden path π for an HMM that emits a string x**. In other words, we would like to **maximize Pr(x, π) among all possible hidden paths π**.  (an optimization problem) <br>  

##### Decoding Problem: 

-  Find an optimal hidden path in an HMM given a string of its emitted symbols.

*Input*: A string x = x1 . . . xn emitted by an HMM (Σ, States, Transition, Emission). <br>
*Output*: A path π that maximizes the probability Pr(x, π) over all possible paths through this HMM. <br>

---

In 1967, Andrew Viterbi used an **HMM-inspired analog of a Manhattan-like grid to solve the Decoding Problem**. 
For an HMM emitting a string of n symbols $x = x_1 . . . x_n$, the nodes in the HMM’s Viterbi graph are divided into $|States|$ rows and $n$ columns.

##### Viterbi graph: 
-  $node(k, i)$ represents state $k$ and the $i$-th emitted symbol. 
-  Each node is connected to all nodes in the column to its right; 
-  the edge connecting $(l, i − 1)$ to $(k, i)$ corresponds to transitioning from state $l$ to state $k$ 
   - (with probability $transition(l,k)$ 
-  and then emitting symbol $x_i$ (with probability $emission_k(x_i)$). 
-  As a result, every path connecting a node in the first column of the Viterbi graph to a node in the final column corresponds to a hidden path $π = π_1 . . . π_n$.

We assign a weight of: $\textit{Weight}_{i}(l,k) = \color{green}{\,\textit{transition}_{\pi_{i-1}, \pi_{i}}} \cdot \color{purple}{\textit{emission}_{\pi_i} (x_i)}$

to the edge connecting $(l, i − 1)$ to $(k, i)$ in the Viterbi graph. Furthermore, we define the **product weight of a path in the Viterbi graph as the product of its edge weights**. For a path from the leftmost column to the rightmost column in the Viterbi graph corresponding to the hidden path π, this product weight is equal to the product of n − 1 terms,

$
\displaystyle\prod_{i=2}^{n} {\color{green}{\,\textit{transition}_{\pi_{i-1},\,\pi_i}}} \cdot {\color{purple}{\textit{emission}_{\pi_i} (x_i)}} = \displaystyle\prod_{i=2}^{n}{\,\textit{Weight}_{i}(l,k)} 
$


￼STOP and Think: How does this expression differ from the formula for Pr(x, π) that we derived in the previous section?

<img src=http://bioinformaticsalgorithms.com/images/HMM/HMM_diagram_three_states.png width="120">
<img src=http://bioinformaticsalgorithms.com/images/HMM/Viterbi_graph_three_states.png width="500">

**Figure**: (Top) The diagram of an HMM with three states (emission/transition probabilities as well as nodes corresponding to emitted symbols are omitted). (Bottom) Given a string of n symbols x = x1 . . . xn emitted by an HMM, Viterbi’s Manhattan is a grid with |States| rows and n columns in which each node is connected to every node in the column to its right. The weight of the edge connecting (l, i − 1) to (k, i) is Weight(l, k) = transitionl, k · emissionk(xi). Unlike the alignment graphs we encountered previously, in which the set of valid directions was restricted to south, east, and southeast edges, every node in a column is connected by an edge to every node in the column to its right in the Viterbi graph.

The only difference between the expression:  $\displaystyle\prod_{i=2}^{n} {\color{green}{\,\textit{transition}_{\pi_{i-1},\,\pi_i}}} \cdot {\color{purple}{\textit{emission}_{\pi_i} (x_i)}} = \displaystyle\prod_{i=1}^{n-1}{\,\textit{Weight}_{i}(l,k)} 
$

and the expression that we obtained for $Pr(x, π)$:   $\displaystyle\prod_{i=1}^{n} \,{\color{green}{\textit{transition}_{\pi_{i-1},\,\pi_i}}} \cdot {\color{purple}{\textit{emission}_{\pi_i} (x_i)}} 
$

is the single factor $transition _{π_0, π_1} · emission_{π_1}(x1),$ which corresponds to transitioning from the initial state $π_0$ to $π_1$ and emitting the first symbol. To model the initial state, we will add a source node source to the Viterbi graph and then connect source to each node (k, 1) in the first column with an edge of weight <br>
$Weight_1(source, k) = transition_{π_0, k} · emission_k(x_1) $. <br>

We will also assume that the HMM has another silent **terminal state** that the HMM enters when it has finished emitting symbols. To model the terminal state, we add a sink node sink to the Viterbi graph and connect every node in the last column to sink with an edge of weight 1 (see figure below).

<img src=http://bioinformaticsalgorithms.com/images/HMM/Viterbi_graph_three_states_source_sink.png width="440">

**Figure**: The Viterbi graph with additional source node (blue) and sink node (red). A path of largest product weight connecting the source to the sink corresponds to an optimal hidden path solving the Decoding Problem.

#### Seems like just a simple dynamic programming problem, as with Needleman-Wunsch alignment (global).

---

<br>

<br>

Every hidden path π in the HMM now corresponds to a path from source to sink in the Viterbi graph with product weight Pr(x, π). Therefore, the Decoding Problem reduces to finding a path in the Viterbi graph of largest product weight over all paths connecting source to sink.

￼Exercise Break: Find the maximum product weight path in the Viterbi graph for the crooked dealer HMM (whose HMM diagram is reproduced below) when x = “HHTT”.  Express your answer as a string of four "F" and "B" symbols.

Note: You may assume that transitions from the initial state occur with equal probability.

<img src=http://bioinformaticsalgorithms.com/images/HMM/HMM_diagram_complete.png width="400">



W

In [90]:
F= 0.5
B= {'heads':0.75, 'tails':0.25}
t = [0.9, 0.1]

heads ='heads'
ff = (0.5)*F
bb = 0.5*B[heads]
print('f1:', ff,'b1:', bb)

#2
b = bb*t[1]*B[heads]
f = ff*t[0]*F
new_f = max(b, f)
print('\nf2:', new_f, 'B?',b>f)

b = bb*t[0]*B[heads]
f = ff*t[1]*F
new_b = max(b, f)
print('b2', new_b, 'B?',b>f)


bb = new_b; ff = new_f

tails = 'tails'
#F3 #B3
b = bb*t[1]*B[tails]
f = ff*t[0]*F
new_f = max(b, f)
print('\nf3', new_f, 'from B?', b>f)
b = bb*t[0]*B[tails]
f = ff*t[1]*F
new_b = max(b, f)
print('b3:', new_b, 'B?',b>f)
bb = new_b;ff = new_f

#4
b = bb*t[1]*B[tails]
f = ff*t[0]*F
new_f = max(b, f)
print('\nF4',new_f,'B?',b>f)

b = bb*t[0]*B[tails]
f = ff*t[1]*F
new_b = max(b, f)
print('b4',new_b,'B?',b>f)



f1: 0.25 b1: 0.375

f2: 0.1125 B? False
b2 0.25312500000000004 B? True

f3 0.050625 from B? False
b3: 0.056953125000000014 B? True

F4 0.022781250000000003 B? False
b4 0.012814453125000003 B? True


So the hidden path that maximizes the probability of that outcome (HHTT), given the T and E matrices is 'FFFF'

---

<br>

#### **The Viterbi algorithm**

<br>

We will apply a dynamic programming algorithm to solve the Decoding Problem. 

First, define $s_{k,i}$ as the product weight of an optimal path (i.e., a path with maximum product weight) from source to the node $(k, i)$. 

The Viterbi algorithm relies on the fact that the first $i − 1$ edges of an optimal path from source to $(k, i)$ must form an optimal path from source to $(l, i − 1)$ for some (unknown) state $l$. This observation yields the following recurrence (perfect substructure):

$
\begin{aligned} s_{k,\,i} & = \displaystyle\max_{\text{all states }l} \left\{ s_{l,\,i-1} \cdot (\text{weight of edge between nodes}(l, i-1)\text{ and }(k, i)) \right\}\\ & = \displaystyle\max_{\text{all states } l} \left\{ s_{l,\,i-1} \cdot \textit{Weight}_i(l,k) \right\}\\ & = \displaystyle\max_{\text{all states } l} \left\{ s_{l,\,i-1} \cdot \color{green}{\textit{transition}_{\pi_{i-1},\,\pi_i}} \cdot {\color{purple}{\textit{emission}_{\pi_i} (x_i)}} \right\} \end{aligned} $




Since source is connected to every node in the first column of the Viterbi graph,

$\begin{aligned} s_{k,\,1} & = s_{\textit{source}} \cdot \left(\text{weight of edge between } \textit{source} \text{ and } (k, 1)\right)\\ & = s_{\textit{source}} \cdot \textit{Weight}_0(\textit{source}, k)\\ & = s_{\textit{source}} \cdot {\color{green}{\textit{transition}_{\textit{source},\,k}}} \cdot {\color{purple}{\textit{emission}_k(x_1)}} \end{aligned} $

 

In order to initialize this recurrence, we set $source$ equal to 1. We can now compute the maximum product weight over all paths from $source$ to $sink$ as

$ s_{\textit{sink}} = \displaystyle\max_{\text{all states } l}s_{l,\,n} $

￼STOP and Think: How can we adapt our algorithm for finding a longest path in a DAG to find a path with maximum product weight?

How fast is the Viterbi algorithm?
We can interpret the Decoding Problem as yet another instance of the Longest Path in a DAG Problem from our work on sequence alignment because the path π maximizing the product weight 

$ \prod_{i=1}^{n} \textit{Weight}_i(\pi_{i-1}, \pi_i) $

also maximizes the **logarithm of this product**, which is equal to:

$ \sum_{i=1}^{n} \log{(\textit{Weight}_i(\pi_{i-1}))} $


Thus, we can substitute the weights of all edges in the Viterbi graph by their logarithms. 
Finding a longest path (i.e. a path maximizing the **sum** of edge weights) in the resulting graph will correspond to a path of maximum product weight in the original Viterbi graph. 
For this reason, the runtime of the Viterbi algorithm, which you are now ready to implement, is **linear in the number of edges in the Viterbi graph**. 
The following exercise shows that the number of these edges is $O(|States|^2·n)$, where n is the number of emitted symbols.

**Exercise Break**: Show that the number of edges in the Viterbi graph of an HMM emitting a string of length $n$ is $|States|^2·(n−1)+2·|States|$.

The viterbi graph has $|states|*n$ nodes: to connect each of |states| nodes to each of |states| nodes the next column requires $|states|^2$ edges and we must do this $(n-1)$ times to create the grid portion of the graph. This accounts for $|states|^2*(n-1)$ edges.

The grid must be connected to source and sink nodes, thus requiring $2*|states|$ edges.

The sum of these is  $|states|^2*(n-1) + 2*|states| $ edges, 

presumably |states| << n, so we ignore the 2*|states| portion for bigO 

<br>

---

<br>

Code Challenge: Implement the Viterbi algorithm solving the Decoding Problem.

Input: A string x, followed by the alphabet from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Output: A path that maximizes the (unconditional) probability Pr(x, π) over all possible paths π.

Note: You may assume that transitions from the initial state occur with equal probability.

Sample Input:

        xyxzzxyxyy
        --------
        x y z
        --------
        A B
        --------
            A	B
        A	0.641	0.359
        B	0.729	0.271
        --------
            x	y	z
        A	0.117	0.691	0.192	
        B	0.097	0.42	0.483

Sample Output:

    AAABBAAAAA

---

<br> 


In [91]:
# VITERBI ALGORITHM: BIOINFO ALGOS- ba10c

import numpy as np

def parse_HMM(lines):
    emission = lines[0].strip()
    alphabet = lines[2].strip().split()
    emission = [int(alphabet.index(em)) for em in emission]
    states = lines[4].strip().split()
    S = len(states)
    T = np.array([line.split()[1:] for line in lines[7:7+S]], float)
    T = np.log(T)
    E = np.array([line.split()[1:] for line in lines[9+S:]], float)
    E = np.log(E)
    return(emission, alphabet, states, T, E)

In [92]:
def viterbi_hiddenpath(emission, T, E, states, alphabet):
    """returns max likelihood hidden path for emission string, given HMM"""
    S = len(states)
    n = len(emission)
    viterbi = np.ones(shape = (S, n)) * -float('inf')
    pointers = [[False for e in range(n)] for s in range(S)] 
    # init first column of viterbi with Pr_emission & 1/States
    for state in range(S):
        viterbi[state][0] = np.log(1/S) + E[state][emission[0]]
        pointers[state][0] = -1
    # Fill viterbi graph using dynamic programming
    for i in range(1,n):
        for state in range(S):
            for prev in range(S):
                p_total = E[state][emission[i]] + T[prev][state] + viterbi[prev][i-1]
                if p_total > viterbi[state][i]:
                    viterbi[state][i] = p_total
                    pointers[state][i] = prev
    # start backtrack from greatest probability in last column of viterbi
    score = -float('inf')
    for state in range(S):
        if viterbi[state][n-1] > score:
            last = state
            score = viterbi[state][n-1]
    path = [last]
    # backtrack to recreate max likelihood hidden_path in reverse
    i = n-1
    while i > 0:
        next = pointers[last][i]
        path.append(next)
        last = next
        i -= 1
    # reverse string to get hidden_path     
    return ''.join(str(states[state]) for state in path[::-1])   

In [93]:
## Sample data1:
with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/10c_test.txt") as f:
    lines = [line.strip() for line in f]
emission, alphabet, states, T, E = parse_HMM(lines)
print('string:', emission,'\nalphabet', alphabet,'states',states,'\n\nE_matrix\n',E, '\n\nT_matrix\n', T)
print('\nsolved ->',viterbi_hiddenpath(emission, T, E, states, alphabet))
print("\tsolution: AAABBAAAAA")

string: [0, 1, 0, 2, 2, 0, 1, 0, 1, 1] 
alphabet ['x', 'y', 'z'] states ['A', 'B'] 

E_matrix
 [[-2.14558134 -0.36961546 -1.65025991]
 [-2.3330443  -0.86750057 -0.72773863]] 

T_matrix
 [[-0.44472582 -1.02443289]
 [-0.31608155 -1.30563646]]

solved -> AAABBAAAAA
	solution: AAABBAAAAA


In [94]:
## extra sample data:

ans = 'AAAAAAAAAAAAAABBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBAAA'

with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/10c_test2.txt") as f:
    lines = [line.strip() for line in f]
    emission, alphabet, states, T, E = parse_HMM(lines)
    print('string:', emission,'\nalphabet', alphabet,'states',states,'\n\nE_matrix\n',E, '\n\nT_matrix\n', T)

print('\nsolved   ->',viterbi_hiddenpath(emission, T, E, states, alphabet))
print("solution ->", ans)
print("== ?", viterbi_hiddenpath(emission, T, E, states, alphabet) == ans)

string: [2, 0, 0, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 2, 0, 2, 2, 2, 1, 2, 2, 0, 0, 0, 2, 0, 0, 1, 1, 1, 2, 0, 1, 0, 2, 1, 0, 1, 0, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 1] 
alphabet ['x', 'y', 'z'] states ['A', 'B'] 

E_matrix
 [[-0.63111179 -1.48722028 -1.42295835]
 [-0.78307189 -1.65025991 -1.04696906]] 

T_matrix
 [[-0.45570632 -1.00512195]
 [-0.94933059 -0.48939034]]

solved   -> AAAAAAAAAAAAAABBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBAAA
solution -> AAAAAAAAAAAAAABBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBAAA
== ? True


In [95]:
## Actual test data:

with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/dataset_26256_7.txt") as f:
    lines = [line.strip() for line in f]
    emission, alphabet, states, T, E = parse_HMM(lines)
    print('string:', emission,'\nalphabet', alphabet,'states',states,'\n\nE_matrix\n',E, '\n\nT_matrix\n', T)

    
ans='BDDDDDDDDDDDBBDDBBBBBBBCBBCBCBBCBBBDDDDDDDDDDDBBBDDDDBBBBCBCBBBBDDDBBBBDDDDDDDBBBCBBCBBBBDDDDDBBDDDD'
print('\nsolved   ->',viterbi_hiddenpath(emission, T, E, states, alphabet))
print("solution ->", ans)
print("== ?", viterbi_hiddenpath(emission, T, E, states, alphabet) == ans)

string: [1, 0, 0, 2, 1, 0, 0, 2, 1, 2, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0, 2, 1, 0, 1, 0, 1, 1, 0, 1, 1, 2, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0, 2, 2, 2, 1, 0, 1, 0, 1, 1, 2, 1, 0, 0, 0, 2, 2, 1, 1, 0, 0, 2, 0, 2, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0, 1, 1, 0, 2, 0, 0] 
alphabet ['x', 'y', 'z'] states ['A', 'B', 'C', 'D'] 

E_matrix
 [[-1.09661429 -1.53711725 -0.79628794]
 [-2.24431618 -0.83471074 -0.77652879]
 [-0.94933059 -1.5945493  -0.89159812]
 [-0.44005655 -2.18925641 -1.41058705]] 

T_matrix
 [[-0.95451194 -1.33560125 -1.47403328 -2.09557092]
 [-1.77195684 -0.9063404  -1.37436579 -1.75446368]
 [-0.80073239 -1.05268336 -2.02495336 -2.65926004]
 [-2.3330443  -1.70374859 -1.5847453  -0.66164851]]

solved   -> BDDDDDDDDDDDBBDDBBBBBBBCBBCBCBBCBBBDDDDDDDDDDDBBBDDDDBBBBCBCBBBBDDDBBBBDDDDDDDBBBCBBCBBBBDDDDDBBDDDD
solution -> BDDDDDDDDDDDBBDDBBBBBBBCBBCBCBBCBBBDDDDDDDDDDDBBBDDDDBBBBCBCBBBBDDDBBBBDDDDDDDBBBCBBCBBBBDDDDDBBDDDD
== ? True


<br>


**Exercise Break**: Apply your solution for the **Decoding Problem to find CG-islands in the first million nucleotides from the human X chromosome** (given in FASTA format). To help you design an HMM for this application, you may assume that transitions from CG-islands to non-CG-islands are rare, occurring with probability 0.001, and that transitions from non-CG-islands to CG-islands are even more rare, occurring with probability 0.0001. How many CG-islands do you find?

In [96]:
import numpy as np

T = np.array([[1-10**-4, 10**-4],[10**-3, 1-10**-3]])

# data from 'finding CG islands' slides
freq0 = np.array([53,79,127,36,37,58,58,41,35,75,81,26,24,105,115,50])/1000
freq1 = np.array([87,58,84,61,67,63,17,63,53,53,63,42,51,70,84,84])/1000
Emission_matrix = np.array([freq0,freq1])

np.save("T_matrix", T)
np.save("E_matrix", Emission_matrix)

In [97]:
E = np.load("E_matrix.npy")
T = np.load("T_matrix.npy")
E, T

(array([[0.053, 0.079, 0.127, 0.036, 0.037, 0.058, 0.058, 0.041, 0.035,
         0.075, 0.081, 0.026, 0.024, 0.105, 0.115, 0.05 ],
        [0.087, 0.058, 0.084, 0.061, 0.067, 0.063, 0.017, 0.063, 0.053,
         0.053, 0.063, 0.042, 0.051, 0.07 , 0.084, 0.084]]),
 array([[9.999e-01, 1.000e-04],
        [1.000e-03, 9.990e-01]]))

In [98]:
States = ["non-island", "CG-island"]
nt = 'ACGT'
alpha = dict(zip([a+b for a in nt for b in nt],range(16)))


In [99]:
from Bio import SeqIO
with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/chrX.txt", 'r') as file:
    xchromosome = SeqIO.read(file, "fasta")

In [100]:
xchromosome

SeqRecord(seq=Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet()), id='chrX', name='chrX', description='chrX', dbxrefs=[])

**finished the viterbi algorithm**  
time for 10d. Let's go.

---

<br>

<br>



Dynamic programming allows us to answer questions about HMMs extending beyond the most likely hidden path. For example, we have already computed the probability Pr(π) of a hidden path π. But what about computing Pr(x), the probability that the HMM will emit a string x?

STOP and Think: Which outcome is more likely in the crooked casino: “HHTT” or “HTHT”? How would you find the most likely sequence of four coin flips?

#### Outcome Likelihood Problem: Find the probability that an HMM emits a given string.

    - Input: A string x = x1 . . . xn emitted by an HMM (Σ, States, Transition, Emission).

    - Output: The probability Pr(x) that the HMM emits x.
    
    
    STOP and Think: To solve the Outcome Likelihood Problem, you can make a slight change to the Viterbi recurrence,

\begin{aligned} s_{k,\,i} = \max_{\text{all states } l} \{s_{l,\,i-1} \cdot \textit{Weight}_i(l, k)\}s \end{aligned}

What is the change?

We have already observed that Pr(x) is equal to the sum of Pr(x, π) over all hidden paths π. However, the number of paths through the Viterbi graph is exponential in the length of the emitted string x, and so we will use dynamic programming to develop a faster approach to compute Pr(x).

We denote the total product weight of all paths from source to node (k, i) in the Viterbi graph as forwardk, i; note that forwardsink is equal to Pr(x). To compute forwardk,i, we will divide all paths connecting source to (k, i) into |States| subsets, where each subset contains those paths that pass through node (l, i − 1) (with product weight forwardl, i−1) before reaching (k, i) for some l between 1 and |States|. Therefore, forwardk, i is the sum of |States| terms,

\begin{aligned} \textit{forward}_{k,\,i} & = \displaystyle\sum_{\text{all states } l }{\textit{forward}_{l,\,i-1} \cdot \left({\text{weight of edge connecting } (l, i-1)\text{ and }(k, i)}\right)}\\ & = \displaystyle\sum_{\text{all states } l } {\textit{forward}_{l,\, i-1} \cdot \textit{Weight}_i(l,k)}\,. \end{aligned} 



Note that the only difference between this recurrence and the Viterbi recurrence,

\begin{aligned} s_{k,\,i} = \displaystyle\max_{\text{all states } l} \left\{ s_{l,\,i-1} \cdot \textit{Weight}_i(l,k) \right\} \end{aligned}

is that the maximization in the Viterbi algorithm has changed into a summation symbol. 
We can now **solve the Outcome Likelihood Problem by computing** $forward_{sink}$, which is equal to:

\begin{aligned} \sum_{\text{all states } k}\textit{forward}_{k,\,n} \end{aligned}


### BA10D: the Outcome Likelihood Problem.
---

**Input**:  A string x, followed by the alphabet from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).


**Output**: The probability Pr(x) that the HMM emits x.

**Note**: You may assume that transitions from the initial state occur with equal probability.

Sample Input:

    xzyyzzyzyy
    --------
    x y z
    --------
    A B
    --------
        A	B
    A	0.303	0.697 
    B	0.831	0.169 
    --------
        x	y	z
    A	0.533	0.065	0.402 
    B	0.342	0.334	0.324

Sample Output:
    1.1005510319694847e-06

#### code

In [101]:
# VITERBI ALGORITHM: BIOINFO ALGOS- ba10d

import numpy as np

def parse_HMM(lines):
    emission = lines[0].strip()
    alphabet = lines[2].strip().split()
    emission = [int(alphabet.index(em)) for em in emission]
    states = lines[4].strip().split()
    S = len(states)
    T = np.array([line.split()[1:] for line in lines[7:7+S]], float)
    E = np.array([line.split()[1:] for line in lines[9+S:]], float)
    return(emission, states, T, E)

In [102]:


def outcome_likelihood(emission, T, E, states): # outcome-likelihood of HMM emitting emissions (sum of all hidden paths)
    """returns likelihood of emission string, given HMM"""

    S = len(states)
    n = len(emission)
    viterbi = np.zeros(shape = (S, n))

    # init first column of viterbi with Pr_emission & 1/States
    for state in range(S):
        viterbi[state][0] = 1/S * E[state][emission[0]]

    # Fill viterbi graph with sums over all incoming edges for ea node
    for i in range(1,n):
        for state in range(S):
            em = E[state][emission[i]]
            for prev in range(S):
                trans = T[prev][state]
                viterbi[state][i] += trans * em * viterbi[prev][i-1]
    return sum(viterbi[s][n-1] for s in range(S))

In [103]:
with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/10d_test.txt") as f:
    lines = [line.strip() for line in f]
    emission, states, T, E = parse_HMM(lines)
    print('string:', emission,'\nstates',states,'\n\nE_matrix\n',E, '\n\nT_matrix\n', T)


outcome_likelihood(emission, T, E, states)

string: [0, 2, 1, 1, 2, 2, 1, 2, 1, 1] 
states ['A', 'B'] 

E_matrix
 [[0.533 0.065 0.402]
 [0.342 0.334 0.324]] 

T_matrix
 [[0.303 0.697]
 [0.831 0.169]]


1.100551031969485e-06

In [104]:
with open("/Users/jasonmoggridge/Dropbox/Rosalind/Coursera_textbook_track/Course6/data/dataset_26257_4.txt") as f:
    lines = [line.strip() for line in f]
    emission, states, T, E = parse_HMM(lines)
    print('string:', emission,'\nstates',states,'\n\nE_matrix\n',E, '\n\nT_matrix\n', T)


outcome_likelihood(emission, T, E, states)

string: [2, 1, 2, 2, 2, 0, 2, 1, 0, 1, 2, 2, 1, 2, 1, 0, 1, 1, 2, 2, 0, 2, 2, 2, 2, 1, 2, 0, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 2, 1, 1, 0, 1, 2, 2, 2, 1, 1, 2, 0, 1, 2, 1, 0, 1, 2, 1, 1, 2, 1, 2, 2, 0, 2, 0, 0, 2, 0, 2, 1, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 0, 0, 1, 2, 0, 2, 2, 2, 1, 1, 0, 1] 
states ['A', 'B', 'C'] 

E_matrix
 [[0.377 0.378 0.245]
 [0.395 0.009 0.596]
 [0.194 0.183 0.623]] 

T_matrix
 [[0.361 0.342 0.297]
 [0.194 0.394 0.412]
 [0.293 0.365 0.342]]


3.105412913344748e-50

Correct ans: 3.105412913344748e-50

done all of BA10 A-D

<br>

#### Most Likely Outcome Problem:
---

Now that we can compute Pr(x) for an emitted string x, a natural question is to find the most likely such string. In the crooked dealer example, this corresponds to finding the most likely sequence of flips over all possible sequences of fair and biased coins that the dealer could use.


Find a most likely string emitted by an HMM.

**Input**: An HMM (Σ, States, Transition, Emission) and an integer n.

**Output**: A most likely string x = x1 . . . xn emitted by this HMM, i.e., a string maximizing the probability Pr(x) that the HMM will emit x.

**Exercise Break:** Solve the Most Likely Outcome Problem (Hint: You may need to build a 3-dimensional version of Viterbi’s Manhattan).

##### My solution:
---
Yes. it would need an extra dimension with a 2d viterbi manhattan for each symbol in the emission alphabet. So we would have a $|states|*|emission seq|*|alphabet|$ 3d grid, and edges from all (i-i, state, symbol) nodes in all graphs to each (i,state,symbol) nodes.

We'd fill out the viterbi grid in the same was as earlier, storing the max weight path to each node in the grid, then walk back along the path, getting the optimal hidden path and their emissions.

---

<br>  <br>  <br>